In [1]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [2]:
import sys
sys.path.append('./vggish')

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
# PARAMETERS
serval_data_folder = "../data/"

# we process train and eval files seperatly
# wav file that are in a '*_trn' folder land in the _trn folder
# wav file that are in a '*_eval' folder land in the _eval folder

In [ ]:
# delete flags if needed
delattr(flags.FLAGS, 'f')
delattr(flags.FLAGS, 'audio_embedding_feature_name')
delattr(flags.FLAGS, 'pca_params')
delattr(flags.FLAGS, 'checkpoint')


In [4]:
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## locatie (handmatig downloaden)
## https://github.com/tensorflow/models/tree/master/research/audioset/vggish
pca_params_vggish_model_folder = serval_data_folder + "vggish_model/models/"
print('Files in vggish model directory: (Expecting vggish_..., youtube_...)')
print(os.listdir(pca_params_vggish_model_folder))

flags.DEFINE_string(
    'pca_params', pca_params_vggish_model_folder + 'vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'checkpoint', pca_params_vggish_model_folder + 'vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

# flags.DEFINE_string(
#    'yt_checkpoint', 'models/youtube_model.ckpt',
#    'Path to the VGGish checkpoint file.')

Files in vggish model directory: (Expecting vggish_..., youtube_...)
['vggish_model.ckpt', 'vggish_pca_params.npz']


In [16]:
wav_samples_custom_path = serval_data_folder + "wav_samples_custom/"
wav_samples_custom_labels_csv = wav_samples_custom_path + "wav_samples_custom_labels.csv"
target_wav_samples_custom_all_enumerated_csv = wav_samples_custom_path + "wav_samples_custom_all_enumerated_and_labeled.csv"

# setup the locations for the tensorflow records
target_folder_tfrecords = serval_data_folder + "tf_records_samples/"
# folder extensions
trn_ext = '_trn'
eval_ext = '_eval'
target_folder_tfrecords_trn = target_folder_tfrecords + trn_ext + '/'
target_folder_tfrecords_eval = target_folder_tfrecords + eval_ext + '/'

In [17]:
# read class labels
wav_samples_class_labels = pd.read_csv(wav_samples_custom_labels_csv, sep=";")
#print(wav_samples_class_labels)

In [18]:
# Enumerate all the available wav files, label them and store a data-frame of the result
def collectAndLabelSamples(full_path_name, label, description, class_name):
    ## Read and rewrite all test files
    files = gfile.Glob(str(full_path_name + "*.wav"))
    
    df = pd.DataFrame(columns=['index', 'label', 'description', 'file_path', 'filename', 'class_name'])
    count = 1
    for file in files:
        df_row = pd.DataFrame({'index':count, 'label':[label], 'description':[description], 'file_path':[file], 'filename':[ntpath.basename(file)], 'class_name':[class_name]})
        df = df.append(df_row, ignore_index = True)
        count = count + 1
    return(df)

In [21]:
## Enumerate all the available wav files, label them and store a data-frame of the result
# we have '_trn' and '_eval' folders now
# we do '_trn' first
folder_ext = eval_ext

df_all_wav_samples = pd.DataFrame(columns=['index', 'label', 'description', 'file_path', 'filename', 'class_name'])
for index, row in wav_samples_class_labels.iterrows():
    if row['enable'] == 1:
        print('Processing record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')
        df_row = collectAndLabelSamples(wav_samples_custom_path + row.folder_name + folder_ext + '/', row.label, row.description, row.folder_name )
        df_all_wav_samples = df_all_wav_samples.append(df_row, ignore_index = True)
    else:
        print('Skipping record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')

Processing record 0; label=1000, description=3 Distance 9mm gun shots, path=3_distance_9mm_shots
Processing record 1; label=1001, description=556 gunshots, path=556_gunshots
Processing record 2; label=1002, description=Brommer Alarm, path=a_brommer_alarms
Processing record 3; label=1003, description=Brommer, path=b_brommers
Processing record 4; label=1004, description=Claxons, path=c_claxons
Processing record 5; label=1005, description=Dichtklappende autodeuren, path=k_dichtklappende_autodeuren
Processing record 6; label=1006, description=Luide mensen (schreeuwen, enz), path=m_luide_mensen_schreeuwen_en_zo
Processing record 7; label=1007, description=Motoren, path=o_motoren
Processing record 8; label=1008, description=Terrasgeluide (stille mensen), path=t_terrasgeluiden
Processing record 9; label=1009, description=Versterkte muziek, path=v_versterkte_muziek


In [22]:
## Write df_all_wav_samples to csv
print(df_all_wav_samples.head())
df_all_wav_samples.to_csv(target_wav_samples_custom_all_enumerated_csv, sep=';')

df_all_wav_samples.shape

  index label               description  \
0     1  1000  3 Distance 9mm gun shots   
1     2  1000  3 Distance 9mm gun shots   
2     3  1000  3 Distance 9mm gun shots   
3     4  1000  3 Distance 9mm gun shots   
4     5  1000  3 Distance 9mm gun shots   

                                           file_path  \
0  ../data/wav_samples_custom/3_distance_9mm_shot...   
1  ../data/wav_samples_custom/3_distance_9mm_shot...   
2  ../data/wav_samples_custom/3_distance_9mm_shot...   
3  ../data/wav_samples_custom/3_distance_9mm_shot...   
4  ../data/wav_samples_custom/3_distance_9mm_shot...   

                        filename            class_name  
0       shot distance9mm.136.wav  3_distance_9mm_shots  
1   shot distance9mm.15_db18.wav  3_distance_9mm_shots  
2  shot distance9mm.380_db12.wav  3_distance_9mm_shots  
3       shot distance9mm.185.wav  3_distance_9mm_shots  
4   shot distance9mm.55_db18.wav  3_distance_9mm_shots  


(6892, 6)

In [23]:
## Read all samples from disk in case you would like to change some by hand.
df_all_wav_samples = []
df_all_wav_samples = pd.read_csv(target_wav_samples_custom_all_enumerated_csv, sep=";")
print(df_all_wav_samples.head().loc[:,'file_path'][1])

../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.15_db18.wav


In [11]:
## Voor alle wav-files in een map, gebeurt het volgende:

## Eerst wordt het wav-file ingelezen en de bijbehorende label(s) opgezocht in een csv-bestand
## Als video-id wordt een willekeurig id'tje gegeven (in dit geval allemaal dezelfde)

## Dan parsen we wav-file naar embeddings: 
# (dit gebeurt door vggish_input.wavfile_to_example aan te roepen)
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
    
## Daarna wordt een sequence example gemaakt (in getSequenceExample) en het als tf-records weggeschreven

In [24]:
## Function that takes examples from wav-file as input and returns a sequence example

def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Prepare a postprocessor to munge the model embeddings.
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)
    
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)

        # Run inference and postprocessing.
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})
        print(embedding_batch.shape)
        if(embedding_batch.shape!=10):
            print("*****************************************************************")

        postprocessed_batch = pproc.postprocess(embedding_batch)
        #print(postprocessed_batch)


        ## Maak labels en video-id voor in de example
        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))

        ## Maak sequence example
        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
        
    return(seq_example)

In [25]:
def encode_wav_files(wav_files, df_labels, output_directory, class_name):
    # setup a writer
    tf_record_writer = tf.python_io.TFRecordWriter(str(output_directory + class_name + '.tfrecord'))
    
    start = time.time()
    count = 0
    for file in wav_files:
        print("[INFO]: Count=", count, "/", len(wav_files), "; Time=", round((time.time() - start)), sep='')
        print("Filename: " + file, sep='')

        ## Fetch labels
        df_sample_labels = df_labels.loc[df_labels.filename == ntpath.basename(file)]
        if len(df_sample_labels) == 0:
            print('[ERROR]: Found sample without labels; filename=', file, sep='')
        else:
            ## .. encode - Part 1
            examples_batch = vggish_input.wavfile_to_examples(file)
            
            if(examples_batch.shape[0]>10):
                nr_sec = examples_batch.shape[0]
                print(nr_sec)
                start = int(math.floor((nr_sec-10)/2))
                print("start: " + str(start))
                end = int(nr_sec-math.ceil((nr_sec-10)/2))
                print("end: " + str(end))
                examples_batch = examples_batch[start:end, :, :]
                print(examples_batch.shape)
            
            ## .. encode - Part 2
            seq_example = getSequenceExample(examples_batch, df_sample_labels.label.unique())
            
            ## .. write to file
            tf_record_writer.write(seq_example.SerializeToString())
            
            ## .. done
            count = count + 1

    tf_record_writer.close()

In [26]:
#  actual tf encoding
start = time.time()
for class_name in df_all_wav_samples.class_name.unique():
    df_class_wav_samples = df_all_wav_samples.loc[df_all_wav_samples.class_name == class_name]
    
    print('Processing: ', class_name, '; sample_count=', len(df_class_wav_samples), sep='')
    #print("[INFO]: Start time: ", round((time.time() - start)))
    # we add the folder extension here
    encode_wav_files(df_class_wav_samples.file_path, df_all_wav_samples, target_folder_tfrecords_eval , class_name)
    #print("[INFO]: End time: ", round((time.time() - start)))
    

Processing: 3_distance_9mm_shots; sample_count=372
[INFO]: Count=0/372; Time=0
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.136.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=1/372; Time=1
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.15_db18.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=2/372; Time=1
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.380_db12.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=3/372; Time=2
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distanc

(5, 128)
*****************************************************************
[INFO]: Count=29/372; Time=21
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.33.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=30/372; Time=22
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.316_db18.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=31/372; Time=23
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.348.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=32/372; Time=23
Filename: ../data/wav_samples_custom/3_distance_

(5, 128)
*****************************************************************
[INFO]: Count=58/372; Time=42
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.316_db6.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=59/372; Time=43
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.384.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=60/372; Time=44
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.346.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=61/372; Time=44
Filename: ../data/wav_samples_custom/3_distance_

(5, 128)
*****************************************************************
[INFO]: Count=87/372; Time=63
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.228.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=88/372; Time=64
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.26_db6.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=89/372; Time=65
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.10.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=90/372; Time=65
Filename: ../data/wav_samples_custom/3_distance_9m

(5, 128)
*****************************************************************
[INFO]: Count=116/372; Time=84
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.55.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=117/372; Time=85
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.351_db6.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=118/372; Time=85
Filename: ../data/wav_samples_custom/3_distance_9mm_shots_eval/shot distance9mm.212.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=119/372; Time=86
Filename: ../data/wav_samples_custom/3_distan

...

INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=994/1044; Time=825
Filename: ../data/wav_samples_custom/t_terrasgeluiden_eval/T.AS20190718.21.161450.10_db6.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=995/1044; Time=826
Filename: ../data/wav_samples_custom/t_terrasgeluiden_eval/T.AS20190718.21.155239.96_db12.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=996/1044; Time=826
Filename: ../data/wav_samples_custom/t_terrasgeluiden_eval/T-AS-roos.002.200120.141228.06.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
******************************

(10, 128)
*****************************************************************
[INFO]: Count=1022/1044; Time=847
Filename: ../data/wav_samples_custom/t_terrasgeluiden_eval/T.AS20191107.11.164056.80.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=1023/1044; Time=848
Filename: ../data/wav_samples_custom/t_terrasgeluiden_eval/T.AS20191107.03.163943.92_db12.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=1024/1044; Time=849
Filename: ../data/wav_samples_custom/t_terrasgeluiden_eval/T.AS20190718.21.155930.19.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=1025/1044; Time=849
Filename: ../data/wav_sa

INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=7/76; Time=5
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.RGracht-0004.154733_db18.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=8/76; Time=6
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.48.191231.144836.40_db18.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=9/76; Time=7
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.18.200102.125837.04_db12.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
**************************************************

INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=36/76; Time=30
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.17.200102.125827.08.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=37/76; Time=31
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.RGracht-0008.172510_db6.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=38/76; Time=31
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.RGracht-0004.154733_db6.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
***************************************************

(10, 128)
*****************************************************************
[INFO]: Count=65/76; Time=53
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.17.200102.125827.08_db12.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=66/76; Time=54
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.RGracht-0004.171313_db6.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=67/76; Time=55
Filename: ../data/wav_samples_custom/v_versterkte_muziek_eval/V.RGracht-0004.171313_db12.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(10, 128)
*****************************************************************
[INFO]: Count=68/76; Time=56
Filename: ../data/wav_samples_custom/v

In [ ]:
## Lees stuk voor stuk alle wav-files in
## Zoek het bijbehorende label op in een csv-bestand
## Bepaal de embeddings

# Prepare a record writer to store the postprocessed embeddings.
## Trainset
if(label == "gunshots"):
    train_tfrecord = str(tfrecord_path + 'gun_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'gun_test.tfrecord')
elif(label == "elephants_angela"):
    train_tfrecord = str(tfrecord_path + 'elephant_angela_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'elephant_angela_test.tfrecord')
elif(label == "background_angela"):
    train_tfrecord = str(tfrecord_path + 'background_angela_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'background_angela_test.tfrecord')
elif(label == "elephants_wenger"):
    train_tfrecord = str(tfrecord_path + 'elephant_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'elephant_wenger_test.tfrecord')
elif(label == "bush_wenger"):
    train_tfrecord = str(tfrecord_path + 'bush_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'bush_wenger_test.tfrecord')

    
    
train_writer = tf.python_io.TFRecordWriter(train_tfrecord)
test_writer = tf.python_io.TFRecordWriter(test_tfrecord)

## Read and rewrite all test files
files = gfile.Glob(str(wavfile_path + "*.wav"))

count = 1
for file in files:
    
    print("file nr: " + str(count))
    print("Filename: " + file)
    
    count += 1
    
    #print("Filename: " + str(file))
   
    ## Find labels belonging to wav-file
    if(label=="gunshots"):
        labels = getLabels(file)
        if(labels[0]==11):
            nr_9mm += 1
        elif(labels[0]==12):
            nr_556 += 1
    elif(label == "elephants_wenger"):
        labels = [13]
        elephants += 1
    elif(label == "bush_wenger"):
        labels = [14]
        bush += 1
    elif(label == "elephants_angela"):
        labels = [15]
        elephants_angela += 1
    elif(label == "background_angela"):
        labels = [16]
        elephants_angela += 1
    
    ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
    #examples_batch = vggish_input.wavfile_to_examples(str(FLAGS.wavfile_path + wav_file))
    examples_batch = vggish_input.wavfile_to_examples(file)
    
    if(examples_batch.shape[0]>10):
        nr_sec = examples_batch.shape[0]
        print(nr_sec)
        start = int(math.floor((nr_sec-10)/2))
        print("start: " + str(start))
        end = int(nr_sec-math.ceil((nr_sec-10)/2))
        print("end: " + str(end))
        examples_batch = examples_batch[start:end, :, :]
        print(examples_batch.shape)
    
    #print("Examples shape: " + str(examples_batch.shape))
   
    seq_example = getSequenceExample(examples_batch, labels)
    
    rand = random.random()
    if rand <= pct_train:
        train_writer.write(seq_example.SerializeToString())
    else:
        test_writer.write(seq_example.SerializeToString())
    
train_writer.close()
test_writer.close()

In [ ]:
print("nr 9mm: " + str(nr_9mm))
print("nr 556: " + str(nr_556))

In [ ]:
print("nr elephants: " + str(elephants))

In [ ]:
print("nr bush: " + str(bush))

In [ ]:
print("nr gun google: " + str(gun_google))